In [1]:
# please ignore warning messages during the installation
!pip install --disable-pip-version-check -q sagemaker==2.35.0
!pip install --disable-pip-version-check -q pandas==1.1.4
!pip install --disable-pip-version-check -q awswrangler==2.7.0
!pip install --disable-pip-version-check -q numpy==1.18.5
!pip install --disable-pip-version-check -q seaborn==0.11.0
!pip install --disable-pip-version-check -q matplotlib===3.3.3

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils 

In [52]:
import pandas as pd
import csv

import boto3
import sagemaker
import pandas as pd
import numpy as np
import botocore

import awswrangler as wr

In [4]:
!aws s3 ls s3://dlai-practical-data-science/data/raw/

2021-04-30 02:21:06    8457214 womens_clothing_ecommerce_reviews.csv


In [5]:
!aws s3 cp s3://dlai-practical-data-science/data/raw/womens_clothing_ecommerce_reviews.csv ./womens_clothing_ecommerce_reviews.csv

download: s3://dlai-practical-data-science/data/raw/womens_clothing_ecommerce_reviews.csv to ./womens_clothing_ecommerce_reviews.csv


In [7]:
%%sh 
ls

ingest_tranform (1).ipynb
womens_clothing_ecommerce_reviews.csv


In [38]:
!aws s3 cp ./womens_clothing_ecommerce_reviews.csv s3://$bucket/data/raw/womens_clothing_ecommerce_reviews.csv

upload: ./womens_clothing_ecommerce_reviews.csv to s3://b13742/data/raw/womens_clothing_ecommerce_reviews.csv


In [8]:
df = pd.read_csv('womens_clothing_ecommerce_reviews.csv', 
                 index_col=0)

In [9]:
df.shape

(23486, 10)

In [10]:
df.head()

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,847,33,"Cute, crisp shirt",If this product was in petite i would get the...,4,1,2,General,Tops,Blouses
1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,1049,50,My favorite buy!,I love love love this jumpsuit. it's fun fl...,5,1,0,General Petite,Bottoms,Pants
4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [11]:
df_transformed = df.rename(columns = {'Review Text': 'review_body',
                                    'Rating': 'star_rating',
                                    'Class Name': 'product_category'})

columns2Drop = ['Clothing ID', 'Age', 'Title', 'Recommended IND', 
               'Positive Feedback Count', 'Division Name', 'Department Name']

df_transformed.drop(columns = columns2Drop,
                    inplace = True)

df_transformed.dropna(inplace=True)

df_transformed.shape

(22628, 3)

In [12]:
df_transformed.head()

,review_body,star_rating,product_category
0,If this product was in petite i would get the...,4,Blouses
1,Love this dress! it's sooo pretty. i happene...,5,Dresses
2,I had such high hopes for this dress and reall...,3,Dresses
3,I love love love this jumpsuit. it's fun fl...,5,Pants
4,This shirt is very flattering to all due to th...,5,Blouses


In [13]:
def to_sentiment(star_rating):
    if star_rating in {1, 2}: # negative
        return -1 
    if star_rating == 3:      # neutral
        return 0
    if star_rating in {4, 5}: # positive
        return 1

# transform star_rating into the sentiment
df_transformed['sentiment'] = df_transformed['star_rating']\
    .apply(lambda star_rating: to_sentiment(star_rating = star_rating) 
)

# drop the star rating column
df_transformed.drop(columns = ['star_rating'],
                    inplace = True)

# remove reviews for product_categories with < 10 reviews
df_transformed = df_transformed.groupby('product_category')\
    .filter(lambda reviews : len(reviews) > 10)[['sentiment', 'review_body', 'product_category']]

df_transformed.shape

(22626, 3)

In [14]:
df_transformed.head()

,sentiment,review_body,product_category
0,1,If this product was in petite i would get the...,Blouses
1,1,Love this dress! it's sooo pretty. i happene...,Dresses
2,0,I had such high hopes for this dress and reall...,Dresses
3,1,I love love love this jumpsuit. it's fun fl...,Pants
4,1,This shirt is very flattering to all due to th...,Blouses


In [15]:
df_transformed.to_csv('./womens_clothing_ecommerce_reviews_transformed.csv', 
                      index=False)

In [65]:
user = 'AmazonSageMaker-ExecutionRole'

config = botocore.config.Config(
    user_agent_extra = f"{user}")

# low-level service client of the boto3 session
sm = boto3.client(service_name='sagemaker', 
                  config=config)

sess = sagemaker.Session(sagemaker_client = sm) 

# sess = sagemaker.Session()                         


bucket = 'b13742'
role = sagemaker.get_execution_role()
region = sess.boto_region_name
account_id = sess.account_id

print('S3 Bucket: {}'.format(bucket))
print('Region: {}'.format(region))
print('Account ID: {}'.format(account_id))

S3 Bucket: b13742
Region: sa-east-1
Account ID: <bound method Session.account_id of <sagemaker.session.Session object at 0x7fd07403b390>>


In [60]:
!aws s3 cp ./womens_clothing_ecommerce_reviews_transformed.csv s3://$bucket/data/transformed/womens_clothing_ecommerce_reviews_transformed.csv

upload: ./womens_clothing_ecommerce_reviews_transformed.csv to s3://b13742/data/transformed/womens_clothing_ecommerce_reviews_transformed.csv


In [66]:
wr.catalog.create_database(
    name='db13742',
    exist_ok=True
)

In [69]:
dbs = wr.catalog.get_databases()

for db in dbs:
    print("Database name: " + db['Name'])

Database name: db13742


In [70]:
wr.catalog.create_csv_table(
    database=db['Name'],
    path='s3://{}/data/transformed/'.format(bucket), 
    table="reviews",    
    columns_types={
        'sentiment': 'int',        
        'review_body': 'string',
        'product_category': 'string'      
    },
    mode='overwrite',
    skip_header_line_count=1,
    sep=','
)

In [73]:
table = wr.catalog.table(database=db['Name'],
                         table='reviews')
table

,Column Name,Type,Partition,Comment
0,sentiment,int,False,
1,review_body,string,False,
2,product_category,string,False,


In [79]:
df_transformed.head(1).T

,0
sentiment,1
review_body,If this product was in petite i would get the...
product_category,Blouses
